In [1]:
import matplotlib.pyplot as plt
import sentinel1decoder
import logging
import pandas as pd
from pathlib import Path

from SARProcessor.focus import RD, get_chunks

In [2]:
inputfile = '/home/roberto/PythonProjects/SSFocus/Data/RAW/IW/ATENE/S1A_IW_RAW__0SDV_20200102T162318_20200102T162351_030624_038242_4CBE.SAFE/s1a-iw-raw-s-vh-20200102t162318-20200102t162351-030624-038242.dat'
filename = Path(inputfile).stem
decoder = sentinel1decoder.Level0Decoder(inputfile, log_level=logging.WARNING)
raw_df = decoder.decode_metadata()
ephemeris = sentinel1decoder.utilities.read_subcommed_data(raw_df)
print("Swath numbers:",raw_df['Swath Number'].unique())
print('filename: ', filename)

Swath numbers: [12 93 10 11 43]
filename:  s1a-iw-raw-s-vh-20200102t162318-20200102t162351-030624-038242


In [3]:
cols = raw_df.columns
print(cols)

Index(['Packet Version Number', 'Packet Type', 'Secondary Header Flag', 'PID',
       'PCAT', 'Sequence Flags', 'Packet Sequence Count', 'Packet Data Length',
       'Coarse Time', 'Fine Time', 'Sync', 'Data Take ID', 'ECC Number',
       'Test Mode', 'Rx Channel ID', 'Instrument Configuration ID',
       'Sub-commutated Ancilliary Data Word Index',
       'Sub-commutated Ancilliary Data Word', 'Space Packet Count',
       'PRI Count', 'Error Flag', 'BAQ Mode', 'BAQ Block Length',
       'Range Decimation', 'Rx Gain', 'Tx Ramp Rate',
       'Tx Pulse Start Frequency', 'Tx Pulse Length', 'Rank', 'PRI', 'SWST',
       'SWL', 'SAS SSB Flag', 'Polarisation', 'Temperature Compensation',
       'Calibration Mode', 'Tx Pulse Number', 'Signal Type', 'Swap Flag',
       'Swath Number', 'Number of Quads'],
      dtype='object')


In [5]:
# for idx in range(len(cols)):
#     if cols[idx] == 'Signal Type':
#         print(idx, cols[idx], raw_df[cols[idx]].unique())
        
# Writing the dictionary to parse the calibration data as:
        # O: echo
        # 1: Noise
        # 2 to 7: NotApplicable
        # 13 to 14: NotApplicable
        # 12: APDNcal
        # 8: Tx Cal
        # 15: TxHCaliso
        # 9: Rx Cal
        # 10: EPDN Cal
        # 11: TA Cal
cal_dict =  {0: 'echo', 1: 'noise', 2: 'notapplicable', 3: 'notapplicable', 4: 'notapplicable', 5: 'notapplicable', 6: 'notapplicable', 7: 'notapplicable', 8: 'txcal', 9: 'rxtcal', 10: 'epdncal', 11: 'tacal', 12: 'apdncal', 13: 'notapplicable', 14: 'notapplicable', 15: 'txhcaliso'}


In [7]:
raw_df['calibration'] = raw_df['Signal Type'].map(cal_dict)

In [15]:
# explore how many rows are there for each calibration type:
print(raw_df['calibration'].value_counts())

echo         51226
apdncal        240
rxtcal         120
epdncal        120
tacal          120
txcal           72
txhcaliso       48
Name: calibration, dtype: int64


In [27]:
# explore how many rows are there for each calibration type:
# print(raw_df['calibration'].value_counts())

# find the indexes when raw_df['calibration'] passes from echo to other:
echo_to_other = raw_df[raw_df['calibration'] != 'echo'].index
print(echo_to_other)

Int64Index([  129,   130,   131,   132,   133,   134,   135,   136,   137,
              138,
            ...
            51833, 51834, 51835, 51836, 51837, 51838, 51839, 51840, 51841,
            51842],
           dtype='int64', length=720)


In [14]:
print(raw_df['calibration'].unique())

# Extracting the calibration data: 'apdncal' 'txcal' 'txhcaliso' 'rxtcal' 'epdncal' 'tacal'
apdncal = raw_df[raw_df['calibration'] == 'apdncal']
txcal = raw_df[raw_df['calibration'] == 'txcal']
txhcaliso = raw_df[raw_df['calibration'] == 'txhcaliso']
rxtcal = raw_df[raw_df['calibration'] == 'rxtcal']
epdncal = raw_df[raw_df['calibration'] == 'epdncal']
tacal = raw_df[raw_df['calibration'] == 'tacal']

# printing the dimensions of the calibration data:
print('apdncal: ', apdncal.shape[0])
print('txcal: ', txcal.shape[0])
print('txhcaliso: ', txhcaliso.shape[0])
print('rxtcal: ', rxtcal.shape[0])
print('epdncal: ', epdncal.shape[0])
print('tacal: ', tacal.shape[0])



['echo' 'apdncal' 'txcal' 'txhcaliso' 'rxtcal' 'epdncal' 'tacal']
apdncal:  240
txcal:  72
txhcaliso:  48
rxtcal:  120
epdncal:  120
tacal:  120


In [ ]:
# make a multi-plot of all the columns in the dataframe:
tmp = raw_df
tmp.plot(subplots=True, figsize=(15, 60))
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
raw_df["Swath Number"].plot()
plt.ylim([8,15])

In [ ]:
try:
    chunks = get_chunks(raw_df)
except:
    print(f"The chunking failed for {inputfile}")

In [ ]:
for idx, chunk in enumerate(chunks):
    focuser = RD(decoder=decoder, raw=chunk, ephemeris=ephemeris)
    img_focused = focuser.process()
    pd.to_pickle(img_focused, f"/home/roberto/PythonProjects/SSFocus/Data/RAW/IW/ROME/Processed/{idx}_chunk_{filename}.pkl")

In [ ]:
%cd /home/roberto/PythonProjects/SSFocus

# Visualize the focus of a single image:

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from SARProcessor.focus import plot_img_focused
import numpy as np
import os
from pathlib import Path

In [ ]:
subswaths = {}
for i in range(32):
    subswaths[i] = pd.read_pickle(f'/home/roberto/PythonProjects/SSFocus/Data/FOCUSED/IW/ATENE/{i}_subswath.pkl')

In [ ]:
A = list(range(1,32,3))
B = list(range(2,32,3))
C = list(range(3,32,3))

img_stacked_A, img_stacked_B, img_stacked_C = [], [], []
for j in range(0,len(A)-1,2):
    img_stacked_A.append(np.hstack((subswaths[A[j]],subswaths[A[j+1]])))
    
# for j in range(0,len(B),2):
#     img_stacked_B.append(np.hstack((subswaths[B[j]],subswaths[B[j+1]])))
    
# for j in range(0, len(C)-1,2):
#     img_stacked_C.append(np.hstack((subswaths[C[j]],subswaths[C[j+1]])))
    

In [ ]:
kwargs = {
    "figsize": (50, 50),
    "vmin": 80,
    "vmax": 400,
    "cmap": "gray",
    "showAxis": False,
}

In [ ]:
im_stack = np.vstack(img_stacked_A[:3])

plot_img_focused(im_stack, **kwargs)




In [ ]:
plot_img_focused(img_stacked_B[0], **kwargs)

In [ ]:
print(img_stacked_B[0].shape)

In [ ]:
top = np.hstack(subswaths[1], subswaths[2])

In [ ]:


for i in range(0,28,3):
    print(subswaths[i].shape)
    plot_img_focused(subswaths[i], **kwargs)
    

In [ ]:
IW1 = list(range(0,28,3))

img_stacked = []
for j in range(1,len(IW1)-1,2):
    print(j)
    img_stacked.append(np.hstack((subswaths[IW1[j]],subswaths[IW1[j+1]])))


In [ ]:
img_v_stacked = np.vstack(img_stacked)

In [ ]:
kwargs = {
    "figsize": (150, 150),
    "vmin": 50,
    "vmax": 700,
    "cmap": "gray",
    "showAxis": False,
}

plot_img_focused(img_v_stacked, **kwargs)

In [ ]:
filename = "s1a-iw-raw-s-vv-20201203t052021-20201203t052053-035517-042709"
basepath = f'{os.environ["SARLENS_DIR"]}/Data/FOCUSED/IW/ROME/{filename}'
chunks = [x for x in Path(basepath).glob('*.pkl')]

print("Number of chunks:", len(chunks))
assert len(chunks) > 0, "No chunks found!"

def check_shapes(IW):
    shape_0 = IW[0].shape
    for ss in IW:
        assert ss.shape == shape_0, "Shapes are not the same!"
        

# check the unique shapes of the chunks
shapes = []
for ss in chunks:
    ii = pd.read_pickle(ss)
    shape_i = ii.shape[1]
    shapes.append(shape_i)

unique = np.unique(shapes, axis=0)
print(unique)

for chunk_n in range(len(chunks)):
    img = pd.read_pickle(basepath + f"/{chunk_n}_subswath_{filename}.pkl")
    print(f"Chunk n. {chunk_n} with shape:", img.shape)
    

# IW1, IW2, IW3 = [], [], []

# for chunk_n in range(0, len(chunks), 3):
#     img = pd.read_pickle(basepath + f"/{chunk_n}_subswath_{filename}.pkl")
#     print(f"Chunk n. {chunk_n} with shape:", img.shape)
#     IW1.append(img)
# check_shapes(IW1)
    
# for chunk_n in range(1, len(chunks), 3):
#     img = pd.read_pickle(basepath + f"/{chunk_n}_subswath_{filename}.pkl")
#     print(f"Chunk n. {chunk_n} with shape:", img.shape)
#     IW2.append(img)
# check_shapes(IW2)

# for chunk_n in range(2, len(chunks), 3):
#     img = pd.read_pickle(basepath + f"/{chunk_n}_subswath_{filename}.pkl")
#     print(f"Chunk n. {chunk_n} with shape:", img.shape)
#     IW3.append(img)
# check_shapes(IW3)


In [ ]:
# stack the images along the rows:
IW1_stacked = np.vstack(IW1)
IW2_stacked = np.vstack(IW2)    
IW3_stacked = np.vstack(IW3)

In [ ]:
kwargs = {
    "figsize": (20, 20),
    "vmin": 0,
    "vmax": 1500,
    "cmap": "gray",
    "showAxis": False,
}

plot_img_focused(stackedImage, **kwargs)

# BAQ

BAQ (Binary Amplitude Quantization) mode is a specific data quantization technique used in Synthetic Aperture Radar (SAR) systems to compress and reduce the amount of data generated during the radar imaging process. SAR systems produce high-resolution images of the Earth's surface by transmitting radar signals and then processing the reflected signals (echoes) from the ground. This process generates a large volume of data, which can be challenging to manage, store, and transmit.

BAQ mode reduces the amount of data by quantizing the amplitude of the received radar echoes. This is achieved by converting the continuous amplitude values of the radar echoes into discrete levels or "bins." Each amplitude value is then represented by a binary code, which requires fewer bits than the original continuous amplitude value.

The main advantage of using BAQ mode in SAR systems is the significant reduction in data size, which simplifies data handling, storage, and transmission. However, quantization can introduce some loss of information, which may affect the image quality to a certain extent. The trade-off between data reduction and image quality depends on the number of quantization levels (or bits) used. More quantization levels result in better image quality but lower data reduction, whereas fewer quantization levels yield higher data reduction but potentially lower image quality.

In summary, BAQ mode is a data quantization technique used in SAR systems to reduce the amount of data generated during the radar imaging process. This technique allows for more manageable data handling, storage, and transmission at the expense of some loss in image quality, depending on the number of quantization levels used.

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df["BAQ Mode"].to_list())
plt.xlim([0,1000])
plt.title('BAQ')
plt.show()

# Processing Time:

In [ ]:
# processor = SARLens(decoder, selection, ephemeris)
# processor.decode_file()
# processor.extract_parameters()
# processor.calculate_wavelength()
# processor.calculate_sample_rates()
# processor.create_fast_time_vector()
# processor.calculate_slant_range()
# processor.calculate_axes()
# processor.calculate_spacecraft_velocity()
# processor.calculate_positions()
# processor.calculate_velocity_and_d()
# processor.process_freq_domain_data()
# processor.apply_range_filter()
# processor.apply_rcmc_filter()
# processor.apply_azimuth_filter()
# processor.plot_img()

# # processor.process()

In [ ]:
processor = SARLens(decoder, selection, ephemeris)

In [ ]:
import time

print('Decode File:')
t0 = time.time()
processor.decode_file()
t1 = time.time()
print('Execution time:', t1 - t0)

print('extract_parameters:')
t2 = time.time()
processor.extract_parameters()
t3 = time.time()
print('Execution time:', t3 - t2)

print('calculate_wavelength:')
t4 = time.time()
processor.calculate_wavelength()
t5 = time.time()
print('Execution time:', t5 - t4)

print('calculate_sample_rates:')
t6 = time.time()
processor.calculate_sample_rates()
t7 = time.time()
print('Execution time:', t7 - t6)

print('create_fast_time_vector:')
t8 = time.time()
processor.create_fast_time_vector()
t9 = time.time()
print('Execution time:', t9 - t8)

print('calculate_slant_range:')
t10 = time.time()
processor.calculate_slant_range()
t11 = time.time()
print('Execution time:', t11 - t10)

print('calculate_axes:')
t12 = time.time()
processor.calculate_axes()
t13 = time.time()
print('Execution time:', t13 - t12)

print('calculate_spacecraft_velocity:')
t14 = time.time()
processor.calculate_spacecraft_velocity()
t15 = time.time()
print('Execution time:', t15 - t14)

print('calculate_positions:')
t16 = time.time()
processor.calculate_positions()
t17 = time.time()
print('Execution time:', t17 - t16)

print('calculate_velocity_and_d:')
t18 = time.time()
processor.calculate_velocity_and_d()
t19 = time.time()
print('Execution time:', t19 - t18)

print('process_freq_domain_data:')
t20 = time.time()
processor.process_freq_domain_data()
t21 = time.time()
print('Execution time:', t21 - t20)


In [ ]:
print('apply_range_filter:')
t0 = time.time()
processor.apply_range_filter()
t1 = time.time()
print('Execution time:', t1 - t0)

print('apply_rcmc_filter:')
t2 = time.time()
processor.apply_rcmc_filter()
t3 = time.time()
print('Execution time:', t3 - t2)

# print('apply_azimuth_filter:')
# t4 = time.time()
# processor.apply_azimuth_filter()
# t5 = time.time()
# print('Execution time:', t5 - t4)

# print('plot_img:')
# t6 = time.time()
# processor.plot_img()
# t7 = time.time()
# print('Execution time:', t7 - t6)

In [ ]:
plt.figure(figsize=(5,15))
plt.imshow(abs(processor.range_doppler_data))
plt.axis(False)
plt.show()

In [ ]:
processor.len_az_line


"""Apply the azimuth filter and create the compressed data."""
processor.az_compressed_data = np.zeros((processor.len_az_line, processor.len_range_line), 'complex')

for az_line_index in range(processor.len_range_line):
       try:
              # d_vector = np.zeros(processor.len_az_line)

              this_az_filter = np.zeros(processor.len_az_line, 'complex')
              # for ii in range(len(processor.az_freq_vals) - 1):  # -1
              for ii in range(len(processor.az_freq_vals) - 1):  # -1
                     this_az_filter[ii] = cmath.exp((4j * cmath.pi * processor.slant_range[ii] * processor.D[ii, az_line_index]) / processor.wavelength)
       
       except IndexError:
              print('Error')
              print('len_processor.az_freq_vals:',len(processor.az_freq_vals))
              print('i:',ii)
              print('slant_range.shape:',processor.slant_range.shape)
              print('slant_range.shape:',processor.D.shape)
              print('az_line_index:',az_line_index)

       result = processor.range_doppler_data[:, az_line_index] * this_az_filter[:]
       result = np.fft.ifft(result)
       processor.az_compressed_data[:, az_line_index] = result

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(abs(processor.az_compressed_data), vmin=0, vmax=1000, origin='lower')
plt.show()

In [ ]:
# SOURCE CODE

iq_array = decoder.decode_file(selection)
print("Raw data shape: ", iq_array.shape)
# Image sizes
len_range_line = iq_array.shape[1]
len_az_line = iq_array.shape[0]

# Extract necessary parameters, see both reference documents.
# All necessary parameters are included into the dataframe
c = sentinel1decoder.constants.speed_of_light
TXPL = selection["Tx Pulse Length"].unique()[0]
print("Tx Pulse length", TXPL)
TXPSF = selection["Tx Pulse Start Frequency"].unique()[0]
print("Tx Pulse Start Freq", TXPSF)
TXPRR = selection["Tx Ramp Rate"].unique()[0]
print("Tx Ramp Rate", TXPRR)
RGDEC = selection["Range Decimation"].unique()[0]
print("Range Decimation", RGDEC)
PRI = selection["PRI"].unique()[0]
print("PRI", RGDEC)
rank = selection["Rank"].unique()[0]
print("Rank", rank)
suppressed_data_time = 320/(8*sentinel1decoder.constants.f_ref)  # see pag. 82 of the reference document (Airbus)
print(suppressed_data_time)
range_start_time = selection["SWST"].unique()[0] + suppressed_data_time
wavelength = c / 5.405e9

# Sample rates
range_sample_freq = sentinel1decoder.utilities.range_dec_to_sample_rate(RGDEC)
range_sample_period = 1 / range_sample_freq
az_sample_freq = 1 / PRI
az_sample_period = PRI

# Fast time vector [s] - defines the time axis along the fast time direction
range_line_num = [i for i in range(len_range_line)]
fast_time = [range_start_time + i * range_sample_period for i in range_line_num]


# Slant range vector - defines R0, the range of the closest approach for each range cell (i.e. the slant range when
# the radar is closest to the target)
slant_range = [(rank * PRI + t) * c / 2 for t in fast_time]

# Axes - defines the frequency axes in each direction after FFT
SWL = len_range_line / range_sample_freq
az_freq_vals = np.arange(-az_sample_freq / 2, az_sample_freq / 2, 1 / (PRI * len_az_line))
range_freq_vals = np.arange(-range_sample_freq / 2, range_sample_freq / 2, 1 / SWL)


# We need two parameters which vary over range and azimuth, so we're going to loop over these once
# D is the cosine of the instantaneous squint angle and is defined by the letter D in most literature
# Define a function to calculate D, then apply it inside the loop
def d(range_freq, velocity):
    return math.sqrt(1 - ((wavelength ** 2 * range_freq ** 2) / (4 * velocity ** 2)))


D = np.zeros((len_az_line, len_range_line))

# Spacecraft velocity - numerical calculation of the effective spacecraft velocity
ecef_vels = ephemeris.apply(lambda x: math.sqrt(
    x["X-axis velocity ECEF"] ** 2 + x["Y-axis velocity ECEF"] ** 2 + x["Z-axis velocity ECEF"] ** 2), axis=1)
velocity_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ecef_vels.unique(),
                           fill_value="extrapolate")
x_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["X-axis position ECEF"].unique(),
                    fill_value="extrapolate")
y_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["Y-axis position ECEF"].unique(),
                    fill_value="extrapolate")
z_interp = interp1d(ephemeris["POD Solution Data Timestamp"].unique(), ephemeris["Z-axis position ECEF"].unique(),
                    fill_value="extrapolate")
space_velocities = selection.apply(lambda x: velocity_interp(x["Coarse Time"] + x["Fine Time"]), axis=1)

x_positions = selection.apply(lambda x: x_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_list()
y_positions = selection.apply(lambda x: y_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_list()
z_positions = selection.apply(lambda x: z_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_list()

a = 6378137 # WGS84 semi major axis
b = 6356752.3142 # WGS84 semi minor axis
velocities = np.zeros((len_az_line, len_range_line))

# Now loop over range and azimuth, and calculate spacecraft velocity and D
for i in range(len_az_line):
    H = math.sqrt(x_positions[i]**2 + y_positions[i]**2 + z_positions[i]**2)
    W = float(space_velocities.iloc[i])/H
    lat = math.atan(z_positions[i] / x_positions[i])
    local_earth_rad = math.sqrt(((a**2 * math.cos(lat))**2 + (b**2 * math.sin(lat))**2) / ((a * math.cos(lat))**2 + (b * math.sin(lat))**2))
    for j in range(len_range_line):
        cos_beta = (local_earth_rad**2 + H**2 - slant_range[j]**2) / (2 * local_earth_rad * H)
        this_ground_velocity = local_earth_rad * W * cos_beta
        velocities[i, j] = math.sqrt(float(space_velocities.iloc[i]) * this_ground_velocity)
        D[i, j] = d(az_freq_vals[i], velocities[i, j])

freq_domain_data = np.zeros((len_az_line, len_range_line), dtype=complex)

for az_index in range(len_az_line):
    range_line = iq_array[az_index, :]
    range_fft = np.fft.fft(range_line)
    freq_domain_data[az_index, :] = range_fft

for range_index in range(len_range_line):
    az_line = freq_domain_data[:, range_index]
    az_fft = np.fft.fft(az_line)
    az_fft = np.fft.fftshift(az_fft)
    freq_domain_data[:, range_index] = az_fft

# Create range filter
num_tx_vals = int(TXPL*range_sample_freq)
tx_replica_time_vals = np.linspace(-TXPL/2, TXPL/2, num=num_tx_vals)
phi1 = TXPSF + TXPRR*TXPL/2
phi2 = TXPRR/2
tx_replica = np.zeros(num_tx_vals, dtype=complex)
for i in range(num_tx_vals):
    tx_replica[i] = cmath.exp(2j * cmath.pi * (phi1*tx_replica_time_vals[i] + phi2*tx_replica_time_vals[i]**2))

range_filter = np.zeros(len_range_line, dtype=complex)
index_start = np.ceil((len_range_line-num_tx_vals)/2)-1
index_end = num_tx_vals+np.ceil((len_range_line-num_tx_vals)/2)-2
range_filter[int(index_start):int(index_end+1)] = tx_replica

range_filter = np.fft.fft(range_filter)
range_filter = np.conjugate(range_filter)

for az_index in range(len_az_line):
    freq_domain_data[az_index, :] = freq_domain_data[az_index, :]*range_filter


rcmc_filt = np.zeros(len_range_line, dtype=complex)
range_freq_vals = np.linspace(-range_sample_freq/2, range_sample_freq/2, num=len_range_line)
for az_index in range(len_az_line):
    rcmc_filt = np.zeros(len_range_line, dtype=complex)
    for range_index in range(len_range_line):
        rcmc_shift = slant_range[0]*((1/D[az_index, range_index])-1)
        rcmc_filt[range_index] = cmath.exp(4j * cmath.pi * range_freq_vals[range_index] * rcmc_shift / c)
    freq_domain_data[az_index, :] = freq_domain_data[az_index, :]*rcmc_filt

range_doppler_data = np.zeros((len_az_line, len_range_line), dtype=complex)
for range_line_index in range(len_az_line):
    ifft = np.fft.ifft(freq_domain_data[range_line_index, :])
    ifft_sorted = np.fft.ifftshift(ifft)
    range_doppler_data[range_line_index, :] = ifft_sorted



In [ ]:
# Create azimuth filter
az_compressed_data = np.zeros((len_az_line, len_range_line), 'complex')

for az_line_index in range(len_range_line):
    d_vector = np.zeros(len_az_line)
    this_az_filter = np.zeros(len_az_line, 'complex')
    print('Azimuth_Filter_shape:',this_az_filter.shape)
    print(len(slant_range))
    print(D.shape)
    print(az_freq_vals.shape)
    print('Azimuth Freq Vals:',len(az_freq_vals)-1)
    for i in range(len(az_freq_vals)-1):  # -1
        this_az_filter[i] = cmath.exp((4j * cmath.pi * slant_range[i] * D[i, az_line_index]) / wavelength)
    result = range_doppler_data[:, az_line_index] * this_az_filter[:]
    result = np.fft.ifft(result)
    az_compressed_data[:, az_line_index] = result

# Plot final image
plt.figure(figsize=(16,100))
plt.title("Sentinel-1 Processed SAR Image")
plt.imshow(abs(az_compressed_data[:,:]), vmin=0, vmax=2000, origin='lower')
plt.xlabel("Down Range (samples)")
plt.ylabel("Cross Range (samples)")
plt.show()